In [1]:
import torch
import torch.nn as nn

from datetime import datetime

from utility import *

In [2]:
LOOKBACK = 61
TRAIN_SPLIT_RATIO = 0.9
columns = [OPEN_INDEX, HIGH_INDEX, LOW_INDEX, CLOSE_INDEX, VOLUME_INDEX]

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [4]:
todays_date = datetime.now().strftime('%Y-%m-%d')
download_monthly_klines('spot', ['BTCUSDT'], ['15m'], '2019-01-01', todays_date, 'train')

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

file already exists! /Users/mahadevmaitri/Downloads/cisc684/Cryptocurrency-Price-Forecasting/train/BTCUSDT/BTCUSDT-15m-2019-01.zip

file already exists! /Users/mahadevmaitri/Downloads/cisc684/Cryptocurrency-Price-Forecasting/train/BTCUSDT/BTCUSDT-15m-2019-02.zip

file already exists! /Users/mahadevmaitri/Downloads/cisc684/Cryptocurrency-Price-Forecasting/train/BTCUSDT/BTCUSDT-15m-2019-03.zip

file already exists! /Users/mahadevmaitri/Downloads/cisc684/Cryptocurrency-Price-Forecasting/train/BTCUSDT/BTCUSDT-15m-2019-04.zip

file already exists! /Users/mahadevmaitri/Downloads/cisc684/Cryptocurrency-Price-Forecasting/train/BTCUSDT/BTCUSDT-15m-2019-05.zip

file already exists! /Users/mahadevmaitri/Downloads/cisc684/Cryptocurrency-Price-Forecasting/train/BTCUSDT/BTCUSDT-15m-2019-06.zip

file already exists! /Users/mahadevmaitri/Downloads/cisc684/Cryptocurrency-Price-Forecasting/train/BTCUSDT/BTCUSDT-15m-2019-07.zip

file already

In [5]:
dataframe = get_data('train', 'BTCUSDT')

In [6]:
x_train, y_train, x_test, y_test = split_data(dataframe, LOOKBACK, TRAIN_SPLIT_RATIO, columns)

In [7]:
print('x_train.shape = {}'.format(x_train.shape))
print('y_train.shape = {}'.format(y_train.shape))
print('x_test.shape = {}'.format(x_test.shape))
print('y_test.shape = {}'.format(y_test.shape))

x_train.shape = (120668, 60, 5)
y_train.shape = (120668,)
x_test.shape = (13407, 60, 5)
y_test.shape = (13407,)


In [8]:
x_train = torch.tensor(x_train, dtype=torch.float)
x_test = torch.tensor(x_test, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.float)

In [9]:
input_dim = x_train.shape[1]
output_dim = 1

In [62]:
class CNN_LSTM(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(CNN_LSTM, self).__init__()
        
        self.conv1 = nn.Conv1d(60, 56, 5)
        self.lstm1 = nn.LSTM(1, 56)
        self.fc1 = nn.Linear(56, 1)
        
    def observe_outputs(self, x):
        x = self.conv1(x)
        print('After conv1: {}'.format(x.size()))
        
        x, _ = self.lstm1(x)
        print('After lstm1: {}'.format(x.size()))
        
        x = self.fc1(x)
        print('After fc1: {}'.format(x.size()))
        
    def forward(self, x):
        x = self.conv1(x)
        x, _ = self.lstm1(x)
        x = self.fc1(x)        

In [63]:
cnn_lstm = CNN_LSTM(input_dim, output_dim).to(device)

x = torch.randn(16, 60, 5).to(device)
with torch.no_grad():
    cnn_lstm.observe_outputs(x)

After conv1: torch.Size([16, 56, 1])
After lstm1: torch.Size([16, 56, 56])
After fc1: torch.Size([16, 56, 1])


In [64]:
def count_parameters(net, all=True):
    return sum(p.numel() for p in net.parameters() if p.requires_grad or all)

num_params = count_parameters(CNN_LSTM(input_dim, output_dim))
print('Total number of parameters: {}'.format(num_params))

Total number of parameters: 30129


In [65]:
def model_train(model, optim, criterion, num_epochs, inputs, actuals):

    hist = np.zeros(num_epochs)
    start_time = datetime.now()

    for t in range(num_epochs):
        inputs = inputs.to(device)
        actuals = actuals.to(device)
        predictions = model(inputs)

        loss = criterion(predictions, actuals)
        if ((t+1) % 10) == 0:
            print("Epoch ", t, "MSE: ", loss.item())
        hist[t] = loss.item()

        optim.zero_grad()
        loss.backward()
        optim.step()
        
    training_time = datetime.now() - start_time
    print("Training time: {}".format(training_time))

    return predictions, hist

In [66]:
def train_test_rmse(actuals_train, predictions_train, actuals_test, predictions_test):
    actuals_train = actuals_train.detach().cpu().numpy()
    predictions_train = predictions_train.detach().cpu().numpy()
    trainScore = np.sqrt(mean_squared_error(actuals_train[:,0], predictions_train[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))

    actuals_test = actuals_test.detach().cpu().numpy()
    predictions_test = predictions_test.detach().cpu().numpy()
    testScore = np.sqrt(mean_squared_error(actuals_test[:,0], predictions_test[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr=0.001)

loss = nn.MSELoss()

y_train_pred, history = model_train(cnn_lstm, optimizer, loss, 1, x_train, y_train)